# Gaussian process example from scikit-learn

This is based on plot_gpr_noisy_targets.ipynb

You can find it here:

http://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpr_noisy_targets.html

Rather than run it all at once, we'll go through it step by step.

First we have to do the necessary imports:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
%matplotlib inline

## Optionally, we set the 'seed' for the random number generator

This makes the sequence of 'random' numbers generated the same every time.

Sometimes you want this. 

For example, if you are developing the program, you want to know that when you change it the changes you see in the results are not just due to different random numbers, but to the changes you made.

You can choose any integer you want; 1 is a popular choice.

In [ ]:
np.random.seed(1)

## Define the data generating function

Since this is an artificial example, we want to know the underlying form of the data.

In real life, you wouldn't know this.

This is a function definition, so when we execute the cell it seems like nothing happens. 

But, further down we can write 'f(3.5)' and Python will know what we mean.

This function has a single argument, and returns a single value.

In [ ]:
def f(x):
    """The function to predict."""
    return x * np.sin(x)

## x coordinates of known values

First we generate a few x coordinates.  

We'll supply the function values at these points, and let the Gaussian process fill in the rest.

In [ ]:
# ----------------------------------------------------------------------
# now the noisy case
X = np.linspace(0.1, 9.9, 20)
X = np.atleast_2d(X).T

print(X.shape)
print(X)

## Creating the y values

We can just call the functio f() that we defined previously using the array X as the argument:

In [ ]:
# Observations and noise
y = f(X).ravel()
print(y)

print(type(y))

## Now create some background noise

Recall that at each point x, the Gaussian process model assumes that we have a mean function f(x) and variance function sigma(x).

First we create a sigma value for each of our data points.

In [ ]:
dy = 0.5 + 1.0 * np.random.random(y.shape)

print(dy.shape)

print(dy)

## Generate the Gaussian noise values

Now we generate a normal(0,sigma) value for each data point, using the sigma values generated in the previous cell.

In [ ]:
noise = np.random.normal(0, dy)

print(noise)

## Finally, generate our 'noisy' measurements y

Add the noise values to the function values to simulate measurements with error.

In [ ]:
y += noise

print(y)

## Generate a mesh on the x axis

We want to estimate values of a function that fits the points we supplied.

While the function is defined at an infinite number of points, we can only work with a finite set.

We'll use the numpy linspace to generate a mesh of values. 

In [ ]:
# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
x = np.atleast_2d(np.linspace(0, 10, 1000)).T

## Check the type and shape of the result

In [ ]:
print(type(x))
print(x.shape)

## Print the first few values

Note that indexing for numpy ndarrays starts at zero, not one.

In [ ]:
print(x[0:15,0])

## Now instantiate a Gaussian process model

Also, display its type

In [ ]:
# Instanciate a Gaussian Process model
kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=(dy / y) ** 2,
                              n_restarts_optimizer=10)

print(type(gp))

In [ ]:
# Fit to data using Maximum Likelihood Estimation of the parameters
gp.fit(X, y)

In [ ]:
# Make the prediction on the meshed x-axis (ask for MSE as well)
y_pred, sigma = gp.predict(x, return_std=True)

## Examine the predicted values object

Note that we only need one subscript for a flattened ndarray

In [ ]:
print(y_pred.shape)
print(y_pred[0:15])

## Examine the sigma object

This represents the uncertainty in the predicted value

Note that it is not the same for each y

In [ ]:
print(sigma.shape)
print(sigma[0:15])

## Graph the original values, the predicted values, and 95% confidence bands

The red dots are the points we supplied.

The blue curve is the mean of the fitted functions at each x value

The blue shaded areas represent the fitted value plus and minus 2 sigmas

The dotted red line is the function that we used to generate the red dots

Note that the means of the Gaussian process functions fit very well, until we get too far past our data. 

In [ ]:
# Plot the function, the prediction and the 95% confidence interval based on
# the MSE
fig = plt.figure()
plt.plot(x, f(x), 'r:', label=u'$f(x) = x\,\sin(x)$')
plt.plot(X, y, 'r.', markersize=10, label=u'Observations')
plt.plot(x, y_pred, 'b-', label=u'Prediction')
plt.fill(np.concatenate([x, x[::-1]]),
         np.concatenate([y_pred - 1.9600 * sigma,
                        (y_pred + 1.9600 * sigma)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.ylim(-10, 20)
plt.legend(loc='upper left')

## What just happened?

Based on twenty data points, we constructed a smooth function that nearly interpolates those points.

In Part 1, we had a smooth function without any superimposed noise, so the curve of the mean function had the appearance of perfectly interpolating the six points, but this is because the estimated error variance at those points is very small.

Again, with a Gaussian process model, we don't need to specify the form of the modeling function.

This is (usually) a big advantage, because in general we don't know the form of the data generating function, and it's best to assume as little as possible.

The Gaussian process model lets the data determine the form of the modeling function.

## The Guassian process model

We are fitting a model of the form:

y = f(x) + e

where f(x) is some function, and e is a normally distributed error term.

These models are very common, for example, linear models (which include regression models, analysis of variance, and analysis of covariance models) have the form:

y = xB + e

where x is a matrix of predictor values and B is a vector of coefficients or 'parameters'

We consider the predictors x to be known, observed quantities.  

An infinite set of possible solution functions is generated by letting the parameter values vary over the real numbers.

However, as big as it is, this set is still confined to a specific type of function.

In Guassian process modeling, f(x) represents an arbitry function that gives us the mean of a Gaussian distribution at each value of x.

The e term is replaced by a function that generates the variance of y at each value of x.  

So the Gaussian process model is:

y = N(mean(x), var(x))

Essentially, the Gaussian process is a model with an infinite number of parameters, a mean and variance for each value of x on the domain.